In [1]:
import numpy as np
import cv2
import PIL
from scipy import misc
from PIL import Image
from pytesseract import image_to_string 
import pytesseract
import argparse
from Queue import Queue

In [2]:
parentChildDict=dict()
direct_paths=[]
hasParentList=[]
image_path = ""
dict_centre_vert=dict()
dict_centres_path=dict()
to_be_removed_paths= []

In [3]:
def negImg(vertForCropping, image_path):#Converting the img to negative and removing text
    image_source = image_path
    gray_img = cv2.imread(image_source)
    
    rect,thresh = cv2.threshold(gray_img,127,255,0)
    newImg = cv2.bitwise_not(thresh)
   
    for vert in vertForCropping:
        #print "whitened rect vert ", vert
        cv2.rectangle(newImg,(vert[0],vert[2]),(vert[1],vert[3]),(255,255,255),-3)

   

    cv2.imwrite("123neg.png",newImg)
    #cv2.imshow("sdd", newImg)
    return newImg

In [4]:
# making all the other rectangles black except the start and end

def iswhite(value):
    #if value == (255,255,255):
    #   return True
    if value != (0,0,0) and value != (127,127,127):
        return True
  
    return False

def getNewVertForCropping(vertForCropping, rect1_centre,  rect2_centre, dict_centre_vert, image_path):
    newVertForCropping=[]
    
    newImg = negImg(vertForCropping, image_path)

    
    
    rect1_x1,  rect1_x2, rect1_y1, rect1_y2 = dict_centre_vert[rect1_centre][0]
    
    rect2_x1,  rect2_x2, rect2_y1,rect2_y2 = dict_centre_vert[rect2_centre][0]

    #rect1_x1, rect1_y1, rect1_x2, rect1_y2 = exclude_p1[0][0], exclude_p1[0][1], exclude_p1[1][0], exclude_p1[1][1]
    #rect2_x1, rect2_y1, rect2_x2, rect2_y2 = exclude_p2[0][0], exclude_p2[0][1], exclude_p2[1][0], exclude_p2[1][1]
  
    
    for vert in vertForCropping:

        #this box which is being considered leave it white
        if vert[0] == rect1_x1 and vert[1] ==rect1_x2 and vert[2] ==rect1_y1 and vert[3] ==rect1_y2:
            pass
        elif vert[0] == rect2_x1 and vert[1] ==rect2_x2 and vert[2] ==rect2_y1 and vert[3] ==rect2_y2:
            pass
        else:
            #make all other boxes black
            cv2.rectangle(newImg,(vert[0]-6,vert[2]-6),(vert[1]+6,vert[3]+6),(0,0,0),-1)
            cv2.imwrite('blackRectNewNeg.png', newImg)
            #print "image written"
            
    
    return True

In [5]:
#code for finding the path b/w start and the end


def getadjacent(n):
    x,y = n
    return [(x-1,y),(x,y-1),(x+1,y),(x,y+1),(x-4,y),(x,y-4),(x+4,y),(x,y+4)]

def BFS(start, end, pixels):

    queue = Queue()
    queue.put([start]) # Wrapping the start tuple in a list

    while not queue.empty():
        
        path = queue.get() 
        #print path
        pixel = path[-1]

        if pixel == end:
            return path

        for adjacent in getadjacent(pixel):
            x,y = adjacent
            
            if iswhite(pixels[x,y]):
                pixels[x,y] = (127,127,127) # see note
                new_path = list(path)
                new_path.append(adjacent)
                queue.put(new_path)

    #print "Queue has been exhausted. No answer was found."



    # invoke: python mazesolver.py <mazefile> <outputfile>[.jpg|.png|etc.]


In [6]:


def ifPathExists(vertForCropping, rect1_centre, rect2_centre, dict_centre_vert, image_path):
    getNewVertForCropping(vertForCropping, rect1_centre, rect2_centre, dict_centre_vert, image_path)
    base_img = Image.open("blackRectNewNeg.png")
    base_pixels = base_img.load()

    path = BFS(rect1_centre, rect2_centre, base_pixels)
    
    path_img = Image.open("blackRectNewNeg.png")
    path_pixels = path_img.load()
    
    
    try:
        for position in path:
            x,y = position
            path_pixels[x,y] = (255,0,0) # red
        

        #print 'Path found between' , rect1_centre, ' and ', rect2_centre
        path_img.save('path.png')

        #setting dict_centres_path
        if (rect1_centre[1]<rect2_centre[1]):
            dict_centres_path[(rect1_centre,rect2_centre)] = path
        else:
            dict_centres_path[(rect2_centre,rect1_centre)] = path


        print rect1_centre,  " <----> ", rect2_centre
        direct_paths.append([rect1_centre,rect2_centre])
        return True
       
    except:
        #print "Processing ...... Finding connections"
        #print 'No path found between' , rect1_centre, ' and ', rect2_centre
        return False
    

In [7]:
#combinations = [ ((5,16),(7,-6)),((1,6), (3,-4)),((4,6), (2,8))]
def sortCombinations(combinations):
    
    initial_sort=[]
    for c in combinations:
        initial_sort.append(sorted(c, key=lambda tup: tup[1]))
    
    sorted_comb = sorted(initial_sort, key=lambda tup: tup[0][1])
   
    return sorted_comb
    
#sortCombinations(combinations)

In [8]:
def getChildren(p):
    for key in parentChildDict:
        if p == key:
            return parentChildDict[p]
    return []

#get parent of a given child

def getParent(c):
    for key in parentChildDict.keys():
        if c in parentChildDict[key]:
            hasParentList.append(c)
            return key
    return "NA"

def alreadyHasParent(c):
    if c in hasParentList:
        return True
    else:
        return False

In [9]:
#find parent and child from direct_paths
#and putting them in dictionay



def setParentChild(dp):
    print "#####################",dp
    y0 = dp[0][1]
    y1 = dp[1][1]

    p1 = getParent(dp[0])
    p2 = getParent(dp[1])
    #print "--------",p1,p2
    
    
    if( p1 != "NA" and p2 != "NA" and p1 == p2):
        print y0, y1, "inside p1==p2"
        return False
    
    else :
    #parent = y1 lava
        #if y0>=y1 and not (p1 != "NA" and p2 != "NA"):
        if y0>=y1:
            print y1, y0, "in else loop"
            #print "PARENT     |     CHILD"
            #print dp[1],dp[0]
            if dp[1] in parentChildDict.keys():
                parentChildDict[dp[1]].append(dp[0])

            else :    
                parentChildDict[dp[1]] =[dp[0]]
            return True

        #elif y1>y0 and not (p1 != "NA" and p2 != "NA"):
        elif y1>y0:
            print y0, y1, "in else loop"
            #print "PARENT     |     CHILD"
            #print dp[0],dp[1]
            if dp[0] in parentChildDict.keys():
                parentChildDict[dp[0]].append(dp[1])

            else:    
                parentChildDict[dp[0]] =[dp[1]]
            return True
    return False

In [10]:
# getting all the combinations

# arab.jpg
uniq = [(375, 553, 1035, 1079), (376, 556, 974, 1016), (377, 556, 896, 954), (376, 556, 839, 878), (875, 1011, 813, 911), (82, 270, 805, 861), (614, 723, 788, 874), (375, 554, 762, 823), (875, 1011, 713, 799), (1061, 1192, 702, 753), (613, 723, 688, 774), (375, 554, 688, 748), (622, 985, 101, 169), (1244, 1373, 688, 760), (312, 512, 600, 675), (74, 274, 700, 775), (75, 274, 901, 974), (799, 1055, 600, 674), (575, 749, 601, 674), (636, 931, 276, 337)]
dict_centre_vert = {(176, 833): [(82, 270, 805, 861)], (668, 831): [(614, 723, 788, 874)], (927, 637): [(799, 1055, 600, 674)], (466, 995): [(376, 556, 974, 1016)], (662, 637): [(575, 749, 601, 674)], (943, 756): [(875, 1011, 713, 799)], (943, 862): [(875, 1011, 813, 911)], (174, 737): [(74, 274, 700, 775)], (466, 858): [(376, 556, 839, 878)], (464, 1057): [(375, 553, 1035, 1079)], (412, 637): [(312, 512, 600, 675)], (466, 925): [(377, 556, 896, 954)], (1308, 724): [(1244, 1373, 688, 760)], (1126, 727): [(1061, 1192, 702, 753)], (668, 731): [(613, 723, 688, 774)], (803, 135): [(622, 985, 101, 169)], (464, 718): [(375, 554, 688, 748)], (174, 937): [(75, 274, 901, 974)], (464, 792): [(375, 554, 762, 823)], (783, 306): [(636, 931, 276, 337)]}
centres = [(464, 1057), (466, 995), (466, 925), (466, 858), (943, 862), (176, 833), (668, 831), (464, 792), (943, 756), (1126, 727), (668, 731), (464, 718), (803, 135), (1308, 724), (412, 637), (174, 737), (174, 937), (927, 637), (662, 637), (783, 306)]

# complex.jpg
# uniq = [(714, 823, 795, 833), (506, 615, 716, 754), (714, 823, 713, 751), (29, 138, 675, 713), (506, 615, 634, 672), (714, 823, 632, 670), (29, 138, 614, 652), (29, 138, 553, 591), (714, 823, 552, 590), (506, 615, 552, 590), (316, 426, 552, 590), (1091, 1201, 503, 541), (880, 990, 503, 541), (28, 138, 491, 529), (316, 426, 471, 509), (1091, 1201, 422, 460), (880, 990, 422, 460), (316, 426, 391, 429), (880, 990, 260, 298), (256, 366, 260, 298), (411, 584, 139, 167), (411, 584, 28, 56), (505, 615, 798, 836), (316, 426, 634, 672), (6, 116, 260, 298), (505, 615, 470, 510), (742, 852, 390, 429), (591, 701, 390, 429), (191, 302, 390, 429), (66, 177, 390, 430), (1091, 1201, 340, 379), (880, 990, 340, 379), (755, 866, 259, 299), (629, 739, 259, 299), (504, 615, 259, 299), (379, 490, 259, 299), (131, 242, 259, 298), (1091, 1202, 573, 613)]
# dict_centre_vert = {(434, 279): [(379, 490, 259, 299)], (1146, 593): [(1091, 1202, 573, 613)], (186, 278): [(131, 242, 259, 298)], (311, 279): [(256, 366, 260, 298)], (121, 410): [(66, 177, 390, 430)], (684, 279): [(629, 739, 259, 299)], (560, 653): [(506, 615, 634, 672)], (1146, 441): [(1091, 1201, 422, 460)], (83, 572): [(29, 138, 553, 591)], (560, 490): [(505, 615, 470, 510)], (371, 571): [(316, 426, 552, 590)], (560, 571): [(506, 615, 552, 590)], (61, 279): [(6, 116, 260, 298)], (83, 510): [(28, 138, 491, 529)], (371, 653): [(316, 426, 634, 672)], (371, 490): [(316, 426, 471, 509)], (935, 359): [(880, 990, 340, 379)], (246, 409): [(191, 302, 390, 429)], (559, 279): [(504, 615, 259, 299)], (560, 735): [(506, 615, 716, 754)], (497, 42): [(411, 584, 28, 56)], (935, 441): [(880, 990, 422, 460)], (810, 279): [(755, 866, 259, 299)], (83, 633): [(29, 138, 614, 652)], (935, 522): [(880, 990, 503, 541)], (1146, 522): [(1091, 1201, 503, 541)], (797, 409): [(742, 852, 390, 429)], (935, 279): [(880, 990, 260, 298)], (768, 732): [(714, 823, 713, 751)], (560, 817): [(505, 615, 798, 836)], (768, 571): [(714, 823, 552, 590)], (768, 651): [(714, 823, 632, 670)], (768, 814): [(714, 823, 795, 833)], (371, 410): [(316, 426, 391, 429)], (497, 153): [(411, 584, 139, 167)], (83, 694): [(29, 138, 675, 713)], (1146, 359): [(1091, 1201, 340, 379)], (646, 409): [(591, 701, 390, 429)]}
# centres = [(768, 814), (560, 735), (768, 732), (83, 694), (560, 653), (768, 651), (83, 633), (83, 572), (768, 571), (560, 571), (371, 571), (1146, 522), (935, 522), (83, 510), (371, 490), (1146, 441), (935, 441), (371, 410), (935, 279), (311, 279), (497, 153), (497, 42), (560, 817), (371, 653), (61, 279), (560, 490), (797, 409), (646, 409), (246, 409), (121, 410), (1146, 359), (935, 359), (810, 279), (684, 279), (559, 279), (434, 279), (186, 278), (1146, 593)]



# hsbc.jpg
# uniq = [(132, 247, 210, 237), (150, 228, 148, 174), (164, 313, 14, 43), (254, 385, 210, 237), (14, 145, 148, 174), (166, 311, 69, 93), (235, 364, 149, 173), (370, 471, 149, 173)]
# dict_centre_vert = {(238, 28): [(164, 313, 14, 43)], (238, 81): [(166, 311, 69, 93)], (189, 223): [(132, 247, 210, 237)], (79, 161): [(14, 145, 148, 174)], (189, 161): [(150, 228, 148, 174)], (299, 161): [(235, 364, 149, 173)], (420, 161): [(370, 471, 149, 173)], (319, 223): [(254, 385, 210, 237)]}
# centres = [(189, 223), (189, 161), (238, 28), (319, 223), (79, 161), (238, 81), (299, 161), (420, 161)]



vertForCropping = uniq 
image_path = "input_images/arab.jpg"
#------------------------
def find_paths(vertForCropping,dict_centre_vert,centres,image_path):
    a = iter(centres)
    import itertools
    combinations=[]
    combinations.extend(itertools.combinations(a, 2))

    # print "-------------------Combinations--------------------"
    # print combinations

    sorted_comb = sortCombinations(combinations)

    # print "---------------Sorted Combinations-----------------"
    # print sorted_comb


    #finding if path is possible b/w all the combinations of rectnagles

    num_of_direct_paths = 0

    print ('\n')
    print ".... Finding relationships between entities ...... "
    
    for c in sorted_comb:

        if (ifPathExists( vertForCropping, c[0], c[1], dict_centre_vert, image_path)):
            num_of_direct_paths = num_of_direct_paths + 1

    print ('\n')
    print "Total number of connections = ", num_of_direct_paths
    
    drawHierarchyLines(image_path)
    
#     print ('\n')
#     print "Number of levels excluding client = ", noOfLvls(clientCentre, centres)-1
    
#     print "DCP-",dict_centres_path
    
    return parentChildDict


In [11]:
def drawHierarchyLines(image_path):
    im = cv2.imread(image_path)
    x = 7
    for k in parentChildDict.keys():

        children = getChildren(k)
        for child in children:
            cv2.line(im,(k[0],k[1]),(child[0],child[1]),((271-x)%255,(5+x)%255,(127+x)%255),10)
        x = x + 13
    cv2.imwrite("output.png",im)

In [12]:
def noOfLvls(centreOfClient,centres):
    max_lvl = 0
    
    for centre in centres:
        lvl = 1
        curr = centre
        while (getParent(curr) != "NA"): 
            lvl = lvl + 1
            curr = getParent(curr) 
        if lvl > max_lvl:
            max_lvl = lvl
 
    return max_lvl


In [13]:
def remove_duplicates(l):
    return list(set(l))

In [14]:
find_paths(vertForCropping,dict_centre_vert,centres,image_path)



.... Finding relationships between entities ...... 
(803, 135)  <---->  (783, 306)
(783, 306)  <---->  (176, 833)
(783, 306)  <---->  (1126, 727)
(783, 306)  <---->  (1308, 724)
(783, 306)  <---->  (412, 637)
(783, 306)  <---->  (174, 737)
(783, 306)  <---->  (174, 937)
(783, 306)  <---->  (927, 637)
(783, 306)  <---->  (662, 637)
(412, 637)  <---->  (464, 1057)
(412, 637)  <---->  (466, 995)
(412, 637)  <---->  (466, 925)
(412, 637)  <---->  (466, 858)
(927, 637)  <---->  (943, 862)
(412, 637)  <---->  (176, 833)
(927, 637)  <---->  (176, 833)
(662, 637)  <---->  (176, 833)
(662, 637)  <---->  (668, 831)
(412, 637)  <---->  (464, 792)
(927, 637)  <---->  (943, 756)
(412, 637)  <---->  (1126, 727)
(927, 637)  <---->  (1126, 727)
(662, 637)  <---->  (1126, 727)
(662, 637)  <---->  (668, 731)
(412, 637)  <---->  (464, 718)
(412, 637)  <---->  (1308, 724)
(927, 637)  <---->  (1308, 724)
(662, 637)  <---->  (1308, 724)
(412, 637)  <---->  (174, 737)
(412, 637)  <---->  (174, 937)
(412, 6

{}

In [ ]:
i_sort = []
tf_sort = []
for k in dict_centres_path.keys():
    i_sort.append(sorted(k, key=lambda tup: tup[1]))

print i_sort
f_sort = sorted(i_sort, key=lambda tup: tup[0][1])

for f in f_sort:
    tf_sort.append(tuple(f))
   

[[(662, 637), (927, 637)], [(783, 306), (1126, 727)], [(464, 718), (464, 1057)], [(783, 306), (662, 637)], [(927, 637), (943, 862)], [(662, 637), (176, 833)], [(466, 925), (466, 995)], [(783, 306), (1308, 724)], [(174, 737), (176, 833)], [(783, 306), (174, 737)], [(466, 995), (464, 1057)], [(783, 306), (412, 637)], [(466, 858), (466, 925)], [(412, 637), (176, 833)], [(1126, 727), (176, 833)], [(783, 306), (927, 637)], [(464, 792), (464, 1057)], [(662, 637), (668, 831)], [(662, 637), (668, 731)], [(464, 718), (466, 925)], [(662, 637), (412, 637)], [(943, 756), (943, 862)], [(174, 737), (174, 937)], [(662, 637), (1308, 724)], [(412, 637), (1308, 724)], [(927, 637), (1308, 724)], [(464, 718), (466, 858)], [(1308, 724), (1126, 727)], [(412, 637), (174, 937)], [(412, 637), (174, 737)], [(1126, 727), (174, 937)], [(1308, 724), (174, 737)], [(464, 792), (466, 858)], [(1308, 724), (174, 937)], [(783, 306), (174, 937)], [(464, 792), (466, 925)], [(927, 637), (943, 756)], [(466, 925), (464, 1057

In [ ]:
tf_sort2 = []
for key in tf_sort:
    dict_big_path_x = dict()
    dict_big_path_y = dict()
    x1 = key[0]
    y1 = key[1]
    
    tf_sort2.append(key)
    
    for key2 in tf_sort2:
        
        x2 = key2[0] 
        y2 = key2[1]
        
        if x1 == x2 and y1 != y2 :
            dict_big_path_x[y2]=dict_centres_path[key2]
            
        elif x1 == y2 and y1 != x2:
            dict_big_path_x[x2]=dict_centres_path[key2]
            
        elif x1 != x2 and y1 == y2:
            dict_big_path_y[x2]=dict_centres_path[key2]
            
        elif x1 != y2 and y1 == x2:
            dict_big_path_y[y2]=dict_centres_path[key2]
            
            
#     print "@",key,"x\n  --------------------"
#     for d in dict_big_path_x:
#         print d
    
#     print "@",key,"y\n "
#     for d in dict_big_path_y:
#         print d
#     print " --------------------"       
    
    
    x_keys = dict_big_path_x.keys()
    y_keys = dict_big_path_y.keys()
    
    for x_key in x_keys:
        for y_key in y_keys:
            if x_key == y_key:
                path1 = dict_big_path_x[x_key]
                path2 = dict_big_path_y[y_key]
                
                
                #as the hop distance we are taking as 4 it might be possible we might be missing some value of x and y
                for i in range (0, len(path1)):
                    x = path1[i][0]
                    y = path1[i][1]
                    
                    path1.append((x+1,y))
                    path1.append((x,y+1))
                    path1.append((x-1,y))
                    path1.append((x,y-1))
                    
                    path1.append((x+2,y))
                    path1.append((x,y+2))
                    path1.append((x-2,y))
                    path1.append((x,y-2))
                    
                    path1.append((x+3,y))
                    path1.append((x,y+3))
                    path1.append((x-3,y))
                    path1.append((x,y-3))
                    
                for i in range (0, len(path2)):
                    x = path2[i][0]
                    y = path2[i][1]
                    path2.append((x+1,y))
                    path2.append((x,y+1))
                    path2.append((x-1,y))
                    path2.append((x,y-1))
                    
                    path2.append((x+2,y))
                    path2.append((x,y+2))
                    path2.append((x-2,y))
                    path2.append((x,y-2))
                    
                    path2.append((x+3,y))
                    path2.append((x,y+3))
                    path2.append((x-3,y))
                    path2.append((x,y-3))
                
                
                spath1 = set(path1)
                spath2 = set(path2)
                
                from itertools import chain
                union = set(chain(spath1,spath2))

                small_path = set(dict_centres_path[key])
                if small_path.issubset(union):
                    print key,"removed because of ",key[0],x_key," and ",y_key,key[1]
                    to_be_removed_paths.append(key)

((662, 637), (412, 637)) removed because of  (662, 637) (783, 306)  and  (783, 306) (412, 637)
((412, 637), (1308, 724)) removed because of  (412, 637) (662, 637)  and  (662, 637) (1308, 724)
((412, 637), (1308, 724)) removed because of  (412, 637) (783, 306)  and  (783, 306) (1308, 724)
((927, 637), (1308, 724)) removed because of  (927, 637) (662, 637)  and  (662, 637) (1308, 724)
((927, 637), (1308, 724)) removed because of  (927, 637) (783, 306)  and  (783, 306) (1308, 724)
((412, 637), (174, 937)) removed because of  (412, 637) (783, 306)  and  (783, 306) (174, 937)


In [ ]:
print "No of Paths before removing dublication ",len(to_be_removed_paths)

In [ ]:
uniq_to_removed_paths = remove_duplicates(to_be_removed_paths)
print "No of Paths after removing dublication  ",len(uniq_to_removed_paths)

In [ ]:
for p in uniq_to_removed_paths:
    print p